In [66]:
import pyaxon
import svtools.report
import json
import pandas as pd

In [67]:
df = pd.read_excel("C:/NLP_Project_1/Failures_(1).xlsx" , engine = 'openpyxl')
df.head()

,Failure Name,Station Name,Stage,Failed Step Name,Bucket Name,Suite,Debug Snapshot,User Notes,Failure Tags,Group,...,5,6,7,8,9,10,11,12,13,14
0,07abab9e,jf04wvaw5337,PreBoot,tpreboot_update_Racepointbeach,"hw_err_msm_corecrashlog_ctrl_haderror,hw_err_m...",GNR-AP-X1_PO_2B_VV,09b2709c-a0af-82da-d61d-dec66ab782bd,NaN,"HW.CFG.ERR, HW.CORR, HW.ERR, HW.KNOWN_ISSUE, S...",Concurrency,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7208d2c8,jf04wvaw5337,PreBoot,hpreboot_installexepackage_1,hw_err_msm_global_status_ctrl_reg_global_viral...,GNR-AP-X1_PO_2B_VV,0ba849d8-7b3b-f571-de4c-0997a0dcdeea,NaN,"HW.CFG.ERR, HW.CORR, HW.ERR, HW.KNOWN_ISSUE, S...",Concurrency,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f3c41875,jf04wvaw5337,PreBoot,hpreboot_updateifwi,hw_err_msm_global_status_ctrl_reg_global_viral...,GNR-AP-X1_PO_2B_VV,a416b26f-70ab-685e-f385-4ad0c1fe9306,NaN,"HW.CFG.ERR, HW.CORR, HW.ERR, HW.KNOWN_ISSUE, H...",Concurrency,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,999d0301,jf04wvaw5026,PreBoot,hpreboot_updatettk3_145,hw_err_msm_global_status_ctrl_reg_general_mca_...,GNR-AP-X1_PO_2B_VV,e656b700-798c-fc35-22aa-c00790793e7a,NaN,"HW.CFG.ERR, HW.CORR, HW.ERR, HW.KNOWN_ISSUE, S...",Memory,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07e9c2c7,jf04wvaw5179,Test,MC_memicals_highmem_1,hw_err_msm_global_status_ctrl_reg_general_mca_...,GNR-AP-X1_PO_2B_VV,ac88f0c3-3105-77dd-b934-b0e0946bc982,NaN,"HW.CFG.ERR, HW.CORR, HW.ERR, HW.KNOWN_ISSUE, H...",Memory,...,['msm_global_status_ctrl_reg.global_viral: Glo...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
code = []
hyperlinks = df['Debug Snapshot']
for hyperlink in hyperlinks:
    if type(hyperlink) == float:
        code.append(hyperlink)
        continue
    x = hyperlink.split('/')
    if x[0] != 'https:':
        code.append(x[0])
        continue
    y = x[4].split('?')
    z = y[1].split('=')
    code.append(z[1])

TypeError: 'str' object is not callable

In [10]:
def get_status_scope_summary(vallog):
    summaries = []
    content_report = "intel-content-report-v1"
    status_scope_report = "intel-svtools-report-v1"
    status_scope_summary_domain = ["HSDES Articles"]
    for domain in status_scope_summary_domain:
        try:
            summaries.append(get_summary(vallog, status_scope_report, domain))
        except Exception as e:
            print(f"Error processing domain {domain}: {e}")
            continue
    return summaries

def get_summary(uuid,report,domain):
    axon = pyaxon.axon.Axon("https://axon.intel.com")
    payload = axon.failure.content.object.get(uuid, report)
    svtools_report_dict = json.loads(payload.decode())
    svreport = svtools.report.Report.from_dict(svtools_report_dict)
    
    hsdes_articles = svtools_report_dict.get(domain)
    print(hsdes_articles)
    
    messages = []
    if isinstance(hsdes_articles, list):
        for article in hsdes_articles:
            messages.append(article.get("message", "No message available"))
    else:
        messages.append(hsdes_articles.get("message", "No message available"))
    
    return messages

print(get_status_scope_summary('b9140941-c27e-6165-38ad-361323edee38'))
'''messages = []
count = 0
for value in code:
    count+=1
    print(f"Count-{count}: {value}")
    messages.append(get_status_scope_summary(value))'''
    

None
Error processing domain HSDES Articles: 'NoneType' object has no attribute 'get'
[]


'messages = []\ncount = 0\nfor value in code:\n    count+=1\n    print(f"Count-{count}: {value}")\n    messages.append(get_status_scope_summary(value))'

In [69]:
from pyaxon import Axon,ServerError
import requests
# Initialize the Axon client
api_token = "3ee33172-5a93-4a26-a761-28147bc2ef1d"
axon = pyaxon.axon.Axon("https://axon.intel.com" , token = api_token)
failure_id = "f7d9fd89-0247-ddae-1f4e-80907618a0b6"
hsdes_links = []
try:
    failure_details = axon.failure.get(failure_id)
    tickets = failure_details['tickets']
    hsdes = tickets['hsdes']
    for hsd in hsdes:
        print(hsd)
        try:
            if hsd['type'] == 'sighting':
                link = f"https://axon.intel.com/app/view/{failure_id}"
                hsd['axon_link'] = link
                hsdes_links.append(hsd)
        except:
            continue
    print(hsdes_links)
except ServerError as e:
    print(f"Error: {e.reason}")
    print(f"Details: {e.details}")

{'link': 'https://hsdes.intel.com/appstore/article/#/14012864796', 'id': '14012864796', 'type': 'test_case'}
{'link': 'https://hsdes.intel.com/appstore/article/#/22016724121', 'id': '22016724121', 'type': 'sighting'}
{'link': 'https://hsdes.intel.com/appstore/article/#/14018772684', 'id': '14018772684'}
[{'link': 'https://hsdes.intel.com/appstore/article/#/22016724121', 'id': '22016724121', 'type': 'sighting', 'axon_link': 'https://axon.intel.com/app/view/f7d9fd89-0247-ddae-1f4e-80907618a0b6'}]


In [6]:
df = pd.DataFrame(messages)
df.to_csv("C:/NLP_Project_1/messages_3.csv" , index = False)
print("Conversion done!!")

Conversion done!!
